# **USE CASE 2.** Sentiment analysis in Flower

## Required libraries and configuration


Import required libraries

In [1]:
import os

import re
import time
import random
import sys

from typing import Dict, Optional, Tuple
from collections import OrderedDict
from typing import List, Tuple

import flwr as fl
import numpy as np
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from flwr.common import Metrics
from tensorflow.keras import models, layers, losses, metrics, optimizers

2023-05-10 10:54:55.409262: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 10:54:55.468725: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-10 10:54:55.469119: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-10 10:54:56.214035: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/jose/flower/venv_flower/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Define some parameters for the simulation, such as the number of clients in the federated scenario, the number of federated rounds, the number of epochs of each client before communicating, and the batch size for training phase

In [2]:
# Some parameters
NUM_CLIENTS = 10 # Number of clients in the federated scenario
NUM_ROUNDS = 10 # Number of learning rounds in the federated computation
NUM_EPOCHS = 5 # Number of epochs that the local dataset is seen each round
BATCH_SIZE = 20 # Batch size for training phase

# Define the seed for random numbers
seed = 10
np.random.seed(seed)
tf.random.set_seed(seed)
tf.keras.utils.set_random_seed(seed)
random.seed(seed)

## Methods for text processing

The following method cleans each tweet by removing urls, since they do not provide any valuable sentiment information. Other unuseful characters are also removed, such as the punctuation marks, whitespaces and numbers. Finally, in order treat the text equally, all the characters are converted to lowercase.

In [3]:
def text_processing(tweet):
    if isinstance(tweet, bytes):
        tweet = tweet.decode('utf-8')
        
    # remove https links
    clean_tweet = re.sub(r'http\S+', '', tweet)
    
    # remove punctuation marks
    punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'
    clean_tweet = ''.join(ch for ch in clean_tweet if ch not in set(punctuation))
    
    # remove numbers
    clean_tweet = re.sub('\d', ' ', clean_tweet)
    
    # remove whitespaces
    clean_tweet = ' '.join(clean_tweet.split())
    
    # convert text to lowercase
    clean_tweet = clean_tweet.lower()
    
    return clean_tweet

## Loading and preparing the input data

The Sentiment140 dataset is not available in TFF, so it needs to be loaded from other source such as the tfds (tensorflow datasets) library. Then, it is adapted to the TFF format, so it can be used to train a model using TFF.

Note: We download the full Sentiment140 dataset, but in order to be able to execute the experiments in reasonable time, we are going to use a portion of it. In this notebook we are selecting just 1% of the data for training and 10% for testing purposes.

In [4]:
sent140 = tfds.load('sentiment140', split=['train[:1%]', 'test[:10%]'])
sent140_train, sent140_test = sent140[0], sent140[1]

Select the text and polarity columns from the original data, and transform it to a dataframe so it can be later used in TFF.

At this point, we do not select the 'user' column because we will create random IID partitions, so the 'user' column is not neccesary. If the user want to try it with non-IID partition, he/she may also keep the 'user' column and use it as client identifier instead of the following random user ID. 

In [5]:
# Transform the data to a dataframe
sent140_train_df = tfds.as_dataframe(sent140_train)[['text', 'polarity']]

# Preprocess and clean text with previously defined method
sent140_train_df['text'] = sent140_train_df['text'].apply(lambda x: text_processing(x))

2023-05-10 10:54:57.851591: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [1]
	 [[{{node Placeholder/_3}}]]
2023-05-10 10:54:57.851868: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
/tmp/ipykernel_37554/1257977802.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [6]:
# Create a random list of ids. Each instance is given a random id, indicating the client where will be distributed
ids_train = [i for i in range(NUM_CLIENTS) for _ in range(len(sent140_train)//NUM_CLIENTS)]
random.Random(seed).shuffle(ids_train)

# Add the id assignment to the dataframe
sent140_train_df['user'] = ids_train
unique_ids_train = list(set(ids_train))

/tmp/ipykernel_37554/2410367150.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sent140_train_df['user'] = ids_train


In [7]:
# Do the same with the test data
sent140_test_df = tfds.as_dataframe(sent140_test)[['text', 'polarity']]
sent140_test_df['text'] = sent140_test_df['text'].apply(lambda x: text_processing(x))
ids_test = [i for i in range(NUM_CLIENTS) for _ in range(len(sent140_test)//NUM_CLIENTS)]
random.Random(seed+1).shuffle(ids_test)
sent140_test_df['user'] = ids_test
unique_ids_test = list(set(ids_test))

2023-05-10 10:55:01.605332: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [1]
	 [[{{node Placeholder/_1}}]]
2023-05-10 10:55:01.605781: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [1]
	 [[{{node Placeholder/_4}}]]
/tmp/ipykernel_37554/347512034.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

For the sake of simplicity, in this notebook we will be dealing with a binary problem. For that purpose, we remove the neutral tweets, so the classifier's aim is to differentiate between positive and negative tweets. 

In [8]:
# Convert into binary problem by deleting neutral opinions
def delete_neutral_ops(df):
    # Remove those tweets whose polarity is 2, i.e., neutral
    df = df.loc[df['polarity']!=2]
    
    # For ease of representation, replace class 4 (i.e., positive sentiment) by 1.
    df['polarity'] = df['polarity'].replace(4, 1)
    
    return df

# Transform both trainin and testing dataframes to a binary problem
sent140_train_df = delete_neutral_ops(sent140_train_df)
sent140_test_df = delete_neutral_ops(sent140_test_df)

/tmp/ipykernel_37554/1513210468.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['polarity'] = df['polarity'].replace(4, 1)


Create the DataLoaders for each client. It creates an array of datasets, one for each client.

In [9]:
# Create DataLoaders for each client
train_data = []
test_data = []
for i in range(NUM_CLIENTS):
    train_data.append(sent140_train_df.loc[sent140_train_df['user']==unique_ids_train[i]])
    test_data.append(sent140_test_df.loc[sent140_test_df['user']==unique_ids_test[i]])

## Create a Deep Learning model

In this case we use a model composed by a pre-trained model from tfhub, as well as dense layers. The pre-trained model is not updated in the example; however, the ``trainable`` parameter can be set to ``True``, so such layers are also fine-tuned in the collaborative training.

Note that any network architecture supported by keras can be used. Besides, Flower also supports PyTorch models, but in contrast to Use Case 1, we use a TFF model here to provide a variety of options.

In [10]:
def create_keras_model():
    # Load pre-trained model
    model = "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2"
    hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=False)
    
    # Set model layers
    model = tf.keras.Sequential()
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(1))
    
    # Compile the model
    model.compile(
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        optimizer="adam",
        metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')]
    )
    
    return model

## Training in the federated scenario

First, we create a FlowerClient class, that includes the information of each simulated client. The class has three methods:
 * `get_parameters`: Get the parameters of the model to send them to the server
 * `fit`: Reveives the model parameters from the server, trains it with local data, and return the updated model parameters to the server
 * `evaluate`: Receives the model from the server and evaluates it with local data

In [11]:
# Class-independent methods to get and set the parameters of a model
def get_parameters(model):
        return model.get_weights()
    
def set_parameters(model, parameters):
    model.set_weights(parameters)

In [12]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, client_train_data, client_test_data):
        # Init the client setting the x (text) and the y (polarity) for both train and testing
        self.model = model
        x_train, y_train = client_train_data['text'].to_numpy(), client_train_data['polarity'].to_numpy()
        x_test, y_test = client_test_data['text'].to_numpy(), client_test_data['polarity'].to_numpy()
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test
        self.train_dataset = tf.data.Dataset.from_tensor_slices(client_train_data.to_dict('list'))

    def get_properties(self, config):
        """Get properties of client."""
        raise Exception("Not implemented")

    def get_parameters(self, config):
        """Get parameters of the local model."""
        return get_parameters(self.model)

    def fit(self, parameters, config):
        """Train parameters on the locally held training set."""
        # Update local model parameters
        set_parameters(self.model, parameters)

        # Get hyperparameters for this round
        batch_size: int = BATCH_SIZE 
        epochs: int = NUM_EPOCHS

        # Train the model using hyperparameters from config
        history = self.model.fit(
            self.x_train,
            self.y_train,
            batch_size,
            epochs,
        )

        # Return updated model parameters and results
        parameters_prime = get_parameters(model=self.model)
        num_examples_train = len(self.x_train)
        results = {
            "loss": history.history["loss"][0],
            "accuracy": history.history["accuracy"][0],
                
        }
        return parameters_prime, num_examples_train, results

    def evaluate(self, parameters, config):
        """Evaluate parameters on the locally held test set."""

        # Update local model with global parameters
        set_parameters(self.model, parameters)

        # Get config values
        steps: int = 10

        # Evaluate global model parameters on the local test data and return results
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, BATCH_SIZE)
        num_examples_test = len(self.x_test)
        
        return loss, num_examples_test, {"accuracy": accuracy}

To simulate the federated scenario in a single machine, the client_fn method allows to create FlowerClients on demand, given the client id.

Note that each client is passed both training and testing local data, so the evaluation over test data is done during the simulation itself.

In [13]:
def client_fn(cid: str) -> FlowerClient:
    # Load model
    model = create_keras_model()

    # Note: each client gets a different train/test data
    client_train_data = train_data[int(cid)]
    client_test_data = test_data[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(model, client_train_data, client_test_data)

In order to show averaged evaluations metrics beyond loss, we should define a method to do that; in this case, the accuracy is weighted averaged. 

In [14]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

Define a training strategy with the weighted FedAvg algorithm.

Then, start the simulation indicating the method to create clients, the number of clients in the simulation, the number of rounds, and the strategy (i.e., the FedAvg strategy to combine local updates). The simulation covers both the federated model training as well as evaluating the model with each local test data.

In [15]:
# Define the configuration during training/fitting the model. It is used later when defining the FedAvg strategy.
def fit_config(server_round: int):
    config = {
        "batch_size": BATCH_SIZE,
        "local_epochs": NUM_EPOCHS
    }
    return config

# Define the configuration during evaluation. It is used later when defining the FedAvg strategy.
def evaluate_config(server_round: int):
    return {"val_steps": 5}

In [16]:
# Define strategy
strategy = fl.server.strategy.FedAvg(
    min_fit_clients=NUM_CLIENTS,
    min_evaluate_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    evaluate_metrics_aggregation_fn=weighted_average,
)

# Start simulation
fl_sim = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
    strategy=strategy,
)

INFO flwr 2023-05-10 10:55:01,729 | app.py:142 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-05-10 10:55:03,815	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
INFO flwr 2023-05-10 10:55:04,937 | app.py:176 | Flower VCE: Ray initialized with resources: {'node:172.25.8.82': 1.0, 'memory': 8634888192.0, 'object_store_memory': 4317444096.0, 'CPU': 12.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.25.8.82': 1.0, 'memory': 8634888192.0, 'object_store_memory': 4317444096.0, 'CPU': 12.0}
INFO flwr 2023-05-10 10:55:04,939 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-05-10 10:55:04,939 | server.py:270 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
(launch_and_g

(launch_and_fit pid=38097) Epoch 1/5


(launch_and_fit pid=38104) 2023-05-10 10:55:11.551881: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(launch_and_fit pid=38101) 2023-05-10 10:55:11.565555: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(launch_and_fit pid=38098) 2023-05-10 10:55:11.951587: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(launch_and_fit pid=38099) 2023-05-10 10:55:12.021296: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


160/160 [==============================] - 1s 1ms/step - loss: 0.6737 - accuracy: 0.6053
(launch_and_fit pid=38097) Epoch 2/5
160/160 [==============================] - 0s 1ms/step - loss: 0.6169 - accuracy: 0.6963
(launch_and_fit pid=38097) Epoch 3/5
160/160 [==============================] - 0s 1ms/step - loss: 0.5795 - accuracy: 0.7159
(launch_and_fit pid=38097) Epoch 4/5
160/160 [==============================] - 0s 2ms/step - loss: 0.5613 - accuracy: 0.7178
(launch_and_fit pid=38097) Epoch 5/5
160/160 [==============================] - 0s 1ms/step - loss: 0.5520 - accuracy: 0.7250


(launch_and_fit pid=38104) 2023-05-10 10:55:13.170043: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 498570752 exceeds 10% of free system memory.
(launch_and_fit pid=38101) 2023-05-10 10:55:13.184109: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 498570752 exceeds 10% of free system memory.
(launch_and_fit pid=38098) 2023-05-10 10:55:13.773500: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 498570752 exceeds 10% of free system memory.
(launch_and_fit pid=38099) 2023-05-10 10:55:13.920770: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 498570752 exceeds 10% of free system memory.
(launch_and_fit pid=38104) 2023-05-10 10:55:15.895741: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 498570752 exceeds 10% of free system memory.


...

(launch_and_evaluate pid=47685) 2023-05-10 11:04:51.907702: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 498570752 exceeds 10% of free system memory.
(launch_and_evaluate pid=47685) 2023-05-10 11:04:52.732978: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 498570752 exceeds 10% of free system memory.
(launch_and_evaluate pid=47683) 2023-05-10 11:04:52.902039: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 498570752 exceeds 10% of free system memory.


1/1 [==============================] - 0s 140ms/step - loss: 0.3829 - accuracy: 0.8571


(launch_and_evaluate pid=47683) 2023-05-10 11:04:53.707012: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 498570752 exceeds 10% of free system memory.
DEBUG flwr 2023-05-10 11:04:54,141 | server.py:179 | evaluate_round 10 received 5 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 5 results and 0 failures
INFO flwr 2023-05-10 11:04:54,142 | server.py:144 | FL finished in 585.2032361740003
INFO:flwr:FL finished in 585.2032361740003


1/1 [==============================] - 0s 107ms/step - loss: 0.2612 - accuracy: 1.0000


INFO flwr 2023-05-10 11:04:54,355 | app.py:198 | app_fit: losses_distributed [(1, 0.4278187516488527), (2, 0.40879114442750025), (3, 0.4083961005273618), (4, 0.4082111190808447), (5, 0.40798475710969223), (6, 0.40515820603621633), (7, 0.4047055173861353), (8, 0.4031940784893538), (9, 0.4035557320243434), (10, 0.4013929861156564)]
INFO:flwr:app_fit: losses_distributed [(1, 0.4278187516488527), (2, 0.40879114442750025), (3, 0.4083961005273618), (4, 0.4082111190808447), (5, 0.40798475710969223), (6, 0.40515820603621633), (7, 0.4047055173861353), (8, 0.4031940784893538), (9, 0.4035557320243434), (10, 0.4013929861156564)]
INFO flwr 2023-05-10 11:04:54,356 | app.py:199 | app_fit: metrics_distributed {'accuracy': [(1, 0.842105264726438), (2, 0.842105264726438), (3, 0.842105264726438), (4, 0.842105264726438), (5, 0.842105264726438), (6, 0.8157894768212971), (7, 0.815789481526927), (8, 0.8157894768212971), (9, 0.815789481526927), (10, 0.842105264726438)]}
INFO:flwr:app_fit: metrics_distributed 

## Evaluation with test data

The evaluation has been done during the simulation. Following, we show the averaged results over test data.
The result of the simulation includes the results on all rounds, so we retrieve those of the last round.

In [17]:
print('Test data, \t Loss={:.4f}, \t Accuracy={:.4f}'.format(fl_sim.losses_distributed[-1][1], fl_sim.metrics_distributed['accuracy'][-1][1]))

Test data, 	 Loss=0.4014, 	 Accuracy=0.8421


In [18]:
fl_sim

History (loss, distributed):
	round 1: 0.4278187516488527
	round 2: 0.40879114442750025
	round 3: 0.4083961005273618
	round 4: 0.4082111190808447
	round 5: 0.40798475710969223
	round 6: 0.40515820603621633
	round 7: 0.4047055173861353
	round 8: 0.4031940784893538
	round 9: 0.4035557320243434
	round 10: 0.4013929861156564
History (metrics, distributed):
{'accuracy': [(1, 0.842105264726438), (2, 0.842105264726438), (3, 0.842105264726438), (4, 0.842105264726438), (5, 0.842105264726438), (6, 0.8157894768212971), (7, 0.815789481526927), (8, 0.8157894768212971), (9, 0.815789481526927), (10, 0.842105264726438)]}

(raylet) [2023-05-10 11:05:04,873 E 37953 37953] (raylet) node_manager.cc:3097: 4 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: aa0acf9ee9e32ddf668348d33a329ec3aa72650aa672c670cc93ed24, IP: 172.25.8.82) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.25.8.82`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
